**Install Dependenices**

****

In [2]:
!pip install openai google-generativeai opencv-python pillow

**Main Code**

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import google.generativeai as genai
from openai import OpenAI
import os

genai.configure(api_key="") # Gemini API key

generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model_genai = genai.GenerativeModel(
    model_name="gemini-1.5-flash-8b",
    generation_config=generation_config,
    system_instruction=(
        "Your name is Asystant AI, and you are here to help. You are made by Tachion Software. "
        "There are several things you should follow: If the user asks a normal question, answer it accordingly. "
        "If the user asks anything about the current image (e.g., \"What do you see?\", \"What is this?\", "
        "\"What am I holding?\", etc.), just say \"moondream\" and don’t say anything else."
    ),
)

chat_session = model_genai.start_chat(history=[])

client = OpenAI(
    base_url="https://api.moondream.ai/v1",
    api_key="" # Moondream API key
)

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});
          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();
          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          div.remove();

          return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

def query_moondream(image_path, message):
    with open(image_path, "rb") as f:
        base64_image = b64encode(f.read()).decode('utf-8')

    response = client.chat.completions.create(
        model="moondream-2B",
        messages=[{
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                {"type": "text", "text": message}
            ]
        }]
    )
    return response

loop = True

while loop:
    message = input("Ask Asystant: ")
    response = chat_session.send_message(message)

    if len(response.text) < 12 and "moondream" in response.text:
        filename = take_photo()
        display(Image(filename))
        moondream_response = query_moondream(filename, message)

        try:
            answer = moondream_response.choices[0].message.content
            print("Answer:", answer)
        except Exception as e:
            print("Error extracting answer:", str(e))

        os.remove(filename)
    else:
        print(response.text)
